In [1]:
import os
import mahotas as mh
from wand.image import Image as WImage
from wand.display import display
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import Image
import pickle
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # 10 x 8 inches
plt.gray()
import seaborn as sns
import theano
import theano.tensor as T
import lasagne
from sklearn.model_selection import train_test_split
import sklearn

:0: FutureWarning: IPython widgets are experimental and may change in the future.


1. Import images with appropriate structure
2. Build Layers
3. Test

http://cs231n.github.io/convolutional-networks/

http://deeplearning.net/software/theano/tutorial/conv_arithmetic.html#transposed-convolution-arithmetic

###Import images with appropriate structure

Might need to build a baby library of examples? These would be a slightly larger size than before.

In [2]:
species = 'cfellah'
size = 100
directory = '/cellar/users/ramarty/Data/ants/version3.0/training/{0}/{1}'.format(species, size)

In [3]:
# image
mh.imread('{0}/images/{1}.pgm'.format(directory, 1))

array([[90, 94, 89, ..., 73, 77, 79],
       [92, 92, 93, ..., 64, 69, 72],
       [94, 94, 92, ..., 63, 62, 65],
       ..., 
       [66, 67, 67, ..., 54, 54, 51],
       [68, 67, 68, ..., 53, 54, 52],
       [67, 67, 68, ..., 56, 54, 55]], dtype=uint8)

In [4]:
# target
np.load('{0}/targets/{1}.npy'.format(directory, 1))

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [5]:
def get_features(number_of_images):
    features = []
    for i in range(number_of_images):
        features.append(mh.imread('{0}/images/{1}.pgm'.format(directory, i)))
    return np.array(features)
def get_targets(number_of_targets):
    targets = []
    for i in range(number_of_targets):
        targets.append(np.load('{0}/targets/{1}.npy'.format(directory, i)))
    return np.array(targets)

In [22]:
X = get_features(14000)
y = get_targets(14000)

In [23]:
len(X)

14000

Split into training and test - maybe use a dictionary like the example?

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [25]:
data = [(X_train, y_train),  (X_test, y_test)]
subsets = ['train', 'test']

In [26]:
dataset = {}
for (subset_data, subset_labels), subset_name in zip(data, subsets):
    # The data is provided in the shape (n_examples, 784)
    # where 784 = width*height = 28*28
    # We need to reshape for convolutional layer shape conventions - explained below!
    subset_data = subset_data.reshape(
    (subset_data.shape[0], 1, 100, 100))
    dataset[subset_name] = {
    # We need to use data matrices of dtype theano.config.floatX
    'X': subset_data.astype(theano.config.floatX),
    # Labels are integers
    'y': subset_labels.astype(theano.config.floatX)}

###Build layers

Layers

In [27]:
# input
input_shape = dataset['train']['X'][0].shape
# the first element is the batch size
l_in = lasagne.layers.InputLayer(
    shape=(None, input_shape[0], input_shape[1], input_shape[2]))

In [28]:
l_in.shape

(None, 1, 100, 100)

In [29]:
# Convolution + pooling

l_conv = lasagne.layers.Conv2DLayer(
     l_in,
     # Here, we set the number of filters and their size.
     num_filters=50, filter_size=(5, 5),
     # lasagne.nonlinearities.rectify is the common ReLU nonlinearity
     nonlinearity=lasagne.nonlinearities.rectify,
     # Use He et. al.'s initialization
     W=lasagne.init.HeNormal(gain='relu'))
    # Other arguments: Convolution type (full, same, or valid) and stride

# Here, we do 2x2 max pooling. The max pooling layer also supports striding
#l_pool1 = lasagne.layers.MaxPool2DLayer(l_conv1, pool_size=(2, 2))

In [30]:
# Dense layer - NINLayer??
#l_dense1 = lasagne.layers.NINLayer(l_conv, num_units=5, W=lasagne.init.GlorotUniform('relu'), b=lasagne.init.Normal())

In [31]:
# TransposedConv2DLayer (also known as fractionally strided convolutions)
# is this different after the pooling? Do I have to do an unpooling step???

l_deconv = lasagne.layers.TransposedConv2DLayer(l_conv, num_filters=l_conv.input_shape[1], 
                                                 filter_size=l_conv.filter_size,
                                                 nonlinearity=lasagne.nonlinearities.rectify)


#Upscale


In [32]:
# Output
l_output = lasagne.layers.NINLayer(
     l_deconv,
     # The number of units in the softmas output layer is the number of classes.
     num_units=1,
     nonlinearity=lasagne.nonlinearities.sigmoid)

In [33]:
print l_output.input_shape
print l_output.output_shape

(None, 1, 100, 100)
(None, 1, 100, 100)


Loss function

In [34]:
predictions = lasagne.layers.get_output(l_output)
targets = T.tensor3('true_output') # unsure if this is right
loss = lasagne.objectives.squared_error(predictions, targets)
loss = lasagne.objectives.aggregate(loss, mode='mean')

Train function

In [35]:
all_params = lasagne.layers.get_all_params(l_output, trainable=True)
updates = lasagne.updates.sgd(loss, all_params, learning_rate=0.001)
train = theano.function([l_in.input_var, targets], loss, updates=updates, allow_input_downcast=True, name='Training')
get_output = theano.function([l_in.input_var], predictions, name='get_output')

In [36]:
#get_output = lasagne.layers.get_output(l_output, deterministic=True)
#predict_fn = theano.function([l_in.input_var], T.argmax(get_output, axis=1))

###Test functionality

In [37]:
# Now, let's train it! We'll chop the training data into mini-batches,
# and compute the validation accuracy every epoch.
BATCH_SIZE = 100
N_EPOCHS = 50
# Keep track of which batch we're training with
batch_idx = 0
# Keep track of which epoch we're on
epoch = 0
while epoch < N_EPOCHS:
    # Extract the training data/label batch and update the parameters with it
    train(dataset['train']['X'][batch_idx:batch_idx + BATCH_SIZE],
    dataset['train']['y'][batch_idx:batch_idx + BATCH_SIZE])
    batch_idx += BATCH_SIZE
    # Once we've trained on the entire training set...
    print batch_idx
    if batch_idx >= dataset['train']['X'].shape[0]:
        # Reset the batch index
        batch_idx = 0
        # Update the number of epochs trained
        epoch += 1
        # Compute the network's on the validation data
        #val_output = get_output(dataset['valid']['X'])
        # The predicted class is just the index of the largest probability in the output
        #val_predictions = np.argmax(val_output, axis=1)
        val_predictions = get_output(dataset['test']['X'])
        # The accuracy is the average number of correct predictions
        # TODO: this line doesn't do what we want... 
        accuracy = np.mean(val_predictions == dataset['test']['y'])
        print("Epoch {} validation accuracy: {}".format(epoch, accuracy))
        # want to print precision and recall here instead

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
Epoch 1 validation accuracy: 0.0
100
200
300
400
500
600
700
800
900


KeyboardInterrupt: 

In [38]:
len(dataset['test']['y'])

4620

In [39]:
dataset['test']['y'][0].sum()

884.0

In [40]:
dataset['test']['y'][0]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [41]:
val_predictions[0][0]

array([[ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ],
       [ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ],
       [ 1.        ,  1.        ,  1.        , ...,  1.        ,
         1.        ,  1.        ],
       ..., 
       [ 1.        ,  1.        ,  1.        , ...,  0.49996219,
         1.        ,  1.        ],
       [ 1.        ,  1.        ,  1.        , ...,  0.49996219,
         0.98363334,  0.49996219],
       [ 1.        ,  1.        ,  0.67586031, ...,  0.49996219,
         0.49996219,  0.49996219]])

In [42]:
print val_predictions[0][0].sum() # the first value is the one that must be iterated!
print val_predictions[0][0].max() # the first value is the one that must be iterated!

9949.15567501
1.0


In [ ]:
cutoff = 0.25
precisions, recalls = [], []
for x in range(132):
    TP, FP, FN = 0, 0, 0
    for i in range(100):
        for j in range(100):
            # val_predictions should == 1 for TP and FP
            if (dataset['test']['y'][x][i][j] > cutoff) & (val_predictions[x][0][i][j] > cutoff):
                TP += 1
            elif (dataset['test']['y'][x][i][j] <= cutoff) & (val_predictions[x][0][i][j] > cutoff):
                FP += 1
            elif (dataset['test']['y'][x][i][j] > cutoff) & (val_predictions[x][0][i][j] <= cutoff):
                FN += 1
    if TP + FP > 0: # & TP + FN > 0:
        precision = TP/float(TP+FP)
        recall = TP/float(TP+FN)
        precisions.append(precision)
        recalls.append(recall)
        print x, precision, recall, dataset['test']['y'][x].sum(), val_predictions[x][0].sum()
        #print val_predictions[x][0].sum(), val_predictions[x][0].max(), val_predictions[x][0].min()
    else:
        print x, "No positives"
        #print val_predictions[x][0].sum(), val_predictions[x][0].max(), val_predictions[x][0].min()

In [34]:
if TP + FP > 0:
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    print precision, recall
else:
    print "No positives"

No positives


###Parameter sweep on cluster

In [1]:
def create_cluster_script(learning, batch):
    
    new_script_file = '/cellar/users/ramarty/Projects/ants/scripts/cluster/version3.0/parameter_sweep.v1.sh'
    
    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/ants/sge-system_files/\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/ants/sge-system_files/\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 1-{0}\n".format(len(learning)))
        out_file.write("#$ -l h_vmem=16G\n")
        out_file.write("#$ -tc 10\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set learnings=({0})\n".format(" ".join([str(x) for x in learning])))
        out_file.write("set batches=({0})\n".format(" ".join([str(x) for x in batch])))
        out_file.write("\n")

        out_file.write("set learning=$learnings[$SGE_TASK_ID]\n")
        out_file.write("set batch=$batches[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        out_file.write("python /cellar/users/ramarty/Projects/ants/scripts/python/version3.0/parameter_sweep.v1.py $learning $batch\n")
        out_file.write("date\n")

In [2]:
learning = [0.001, 0.01, 0.1, 1, 10, 100, 1000]*5
batch = [1]*5 + [20]*5 + [100]*5 + [500]*5 + [1000]*5

In [3]:
create_cluster_script(learning, batch)